In [43]:
import datetime
import json
import requests
import sys
import time
import geopandas
from path import Path
import os
import pandas
import pyxlsb
import urllib.request

In [44]:
# path configuration
working_directory = Path.getcwd()
outputs_path = 'OneDrive - HDR, Inc/Data_Science/MODOT_RR/outputs'
# spreadsheet path
xlsx_path = 'General - DL10384336_MoDOT_MNA Freight Crossings/analysis/MoDOT MNA xing analysis 2023_10_09.xlsb'

In [52]:
# import crossings path
from crossings_list import crossings_list

In [ ]:
# list sheet names
pandas.ExcelFile(xlsx_path).sheet_names

In [149]:
# read in spreadsheet
xlsx = pandas.read_excel(xlsx_path,sheet_name = 'MNA Blocked Crossings')

### Read in base data approach if already collected

In [150]:
display(
     xlsx.head(),
     xlsx.shape
)

,Crossing ID,City,State,Street,County,Railroad,Date/Time,Duration,Reason,Immediate Impacts,Additional Comments
0,434842X,JOPLIN,MO,NEWMAN RD,Jasper,MNA,44542.875000,31-60 minutes,A stationary train,NaN,NaN
1,434842X,JOPLIN,MO,NEWMAN RD,Jasper,MNA,44542.873611,31-60 minutes,A stationary train,NaN,This train has been here for 30+ minutes going...
2,435217R,CARTHAGE,MO,CO RD 118,JASPER,MNA,44906.495139,16-30 minutes,A moving train,NaN,Traffic congestion
3,443003U,HARRISONVILLE,MO,E 215TH STREET,CASS,MNA,45143.478472,16-30 minutes,No train was present but the lights and/or gat...,"Pedestrians were observed climbing on, over, o...",NaN
4,443009K,HARRISONVILLE,MO,INDEPENDENCE ST,Cass,MNA,44741.639583,16-30 minutes,A stationary train,NaN,NaN


(17, 11)

In [151]:
# convert datetime
xlsx['Date/Time'] = pandas.to_datetime(xlsx['Date/Time'], unit='D', origin='1899-12-30').dt.strftime('%m/%d/%Y')

##### store and append to other datasets

In [158]:
%store xlsx

# export
xlsx.to_csv('blocked_crossings.csv')

Stored 'xlsx' (DataFrame)


### API Approach - note the API currently doesn't support the date range initially used

In [90]:
# Define the number of items to retrieve per API call
items_per_call = 10
current_skip = 0
total_items_to_retrieve = 200
state_name = 'MO'  # Set the state name you want to filter for

data_list = []

while current_skip < total_items_to_retrieve:
    url = f"https://www.fra.dot.gov/blockedcrossings/api/incidents?page={current_skip // items_per_call + 1}&pageSize={items_per_call}"
    
    with urllib.request.urlopen(url) as response:
        if response.getcode() == 200:
            # Read the response content before converting to JSON
            content = response.read()
            
            # Filter and process the content
            data = json.loads(content)
            filtered_data = [item for item in data['items'] if item['state'] == state_name]
            data_list.extend(filtered_data)

            current_skip += items_per_call

        else:
            print("Failed to fetch data. Status code:", response.getcode())
            break

df = pandas.DataFrame(data_list)

In [77]:
# view df 
display(
     df.head(),
     df.shape
)

,crossingID,reason,city,state,street,railroad,dateTime,duration,created,county,...,longitude,countyCode,stateCode,cityCode,stateCountyFips,communityImpacts,otherImpact,createdBy,modifiedBy,modified
0,063293E,A stationary train,NORTH KANSAS CITY,MO,E 12AVE,BNSF,2023-11-13T15:56:00,1-2 hours,2023-11-13T15:57:10.677,CLAY,...,-94.573303,29047,29,290475770,2929047,"false,false,true",Two trains blocking business district both are...,None,None,2023-11-13T15:57:10.677
1,441933V,A stationary train,INDEPENDENCE,MO,Sterling Avenue,UP,2023-11-10T14:43:00,2-6 hours,2023-11-10T14:44:31.603,JACKSON,...,-94.445863,29095,29,290953940,2929095,"true,true,true",Business entrances and exit being blocked. Sem...,None,None,2023-11-10T14:44:31.603
2,422931F,A stationary train,KANSAS CITY,MO,23rd Street,UP,2023-11-10T14:42:00,1-2 hours,2023-11-10T14:43:23.12,JACKSON,...,-94.494283,29095,29,290954120,2929095,"true,true,false",Truck route blocked with no exit.,None,None,2023-11-10T14:43:23.12
3,669820Y,No train was present but the lights and/or gat...,ASH GROVE,MO,STATE SEC RD,BNSF,2023-11-08T15:00:00,31-60 minutes,2023-11-08T15:17:57.657,GREENE,...,-93.431288,29077,29,290770370,2929077,"false,false,false",,None,None,2023-11-08T15:17:57.657
4,673278L,A stationary train,REPUBLIC,MO,HINES ST,BNSF,2023-11-07T23:30:00,31-60 minutes,2023-11-08T14:11:27.763,GREENE,...,-93.469101,29077,29,290776710,2929077,"false,false,true",Traffic was completely gridlocked on either si...,None,None,2023-11-08T14:11:27.763


(10, 21)

In [95]:
# imagine you don't know the blocked crossings - filtetr from known list of crossings and then query data from FRA
filtered_df = df[df.crossingID.isin(crossings_list)]